# Stations Over Time
This notebook generates a timelapse of station availability over time. The `stations.csv` file is manually populated, and used to generate a `stations-over-time.csv` file. 

In [ ]:
# import conjugate_map.conj_calc as conj
import datetime as dt
import pandas as pd
import plotly.express as px

import geopandas
# import gpxpy
# import gpxpy.gpx
# import json

import logging
logger = logging.getLogger(__name__)

In [ ]:
stations = pd.read_csv('stations_AntarcticdatacoverageFIgure.csv')
stations["End"] = stations["Estimated End"]
stations["End"] = stations["End"].fillna(2025) # assume that all stations will last unless noted otherwise
stations["Start"] = stations["Start"].fillna(1980) # assume that all stations will last unless noted otherwise
stations = pd.DataFrame(stations)


# Restrict to only the networks of interest:
# stations = stations[stations.Network.isin(["MACCS", "AALPIP"])]
# pd.unique(stations.Institution)

In [ ]:
stations

In [ ]:
# Preallocate:
stations_over_time = pd.DataFrame()


for year in [2000, 2017, 2024, 2032]:
    try:
        print("Computing for " + str(year) + "...................................................................")
        # run conjcalc for the year of interest
        # data = conj.conjcalc(stations, latname = 'GLAT', lonname = 'GLON', dtime = dt.datetime(year, 1, 1), mode='N2S')
        data = stations
        # add year column to table
        data = pd.DataFrame(data)
        data['Year'] = year
        # append table to big table
        # stations_over_time.concat(data)
        stations_over_time = pd.concat([stations_over_time, data])
        print('Appending to main table.........................................')
        print(stations_over_time.shape)
        # stations_over_time = pd.concat(data)
        
    except Exception as e:
        print(e)


# write DataFrame to an excel sheet 
stations_over_time
stations_over_time.to_csv('stations-over-time.csv')
stations_over_time.head()

I'm attaching an updated spreadsheet of instrument locations with a new column for estimated end date. I only include the stations in the spreadsheet that I'd like to include in the Figures - all others are deleted. The included locations are a little arbitrary, but they correspond primarily to locations directly supported by NSF/USAP, international collaborations involving NSF/USAP, or figures/data products described earlier in the report.

When you have the chance, it would be great if you could make a map for (1) 2017 [all sites and stations in the spreadsheet are active], (2) 2025 [several AGO and BAS sites decommissioned, but most sites still active], and (3) 2032 [everything is gone except for the stations].

For this, it would be great if you could have different symbols for deep field magnetometers (AALPIP, AGO, BAS LPM), GNSS (or GNSS/seismic - includes POLENET and UKANET), and stations (may include both magnetometer and GNSS). It would also be nice to have a few lines of constant geographic latitude/longitude, just for reference.

In [ ]:
stations_over_time.shape

In [ ]:
import plotly.express as px

In [ ]:
# # Read .csv file:
# stations_over_time = pd.read_csv('stations-over-time.csv')

# TODO: If year < start or > end, delete line.
stations_over_time = stations_over_time[stations_over_time.Year <stations_over_time.End]
stations_over_time = stations_over_time[stations_over_time.Year >stations_over_time.Start]

# Create the animation
world_map = px.scatter_geo(stations_over_time.reset_index(), lon='GLON', lat='GLAT', 
                           color = 'Network',
                           symbol = 'Network',
                           # size = 'Year',  
                           animation_frame='Year', animation_group='Year', projection='orthographic', hover_name='Name', hover_data = ['ID', 'Network'], 
                          # symbol = 'Category'
                          )
# world_map = px.scatter_geo(world, lon='long', lat='lat', size='mass', color='orange', animation_frame='year', animation_group='group', projection='orthographic', hover_name='name')
world_map.update_layout(title='Conjugate Points Over Time', title_x=0.5, title_font=dict(size=10))

# Display the animation
world_map.show()
world_map.write_html("output/animated-map.html")